In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

In [3]:
deaths = pd.read_csv(deaths_url)

In [4]:
deaths = pd.melt(deaths, id_vars=['Country/Region', 'Lat', 'Long', 'Province/State'], 
                var_name = 'Date', 
                value_name = 'Deaths'
       )

In [5]:
deaths.head()

,Country/Region,Lat,Long,Province/State,Date,Deaths
0,Afghanistan,33.0000,65.0000,NaN,1/22/20,0
1,Albania,41.1533,20.1683,NaN,1/22/20,0
2,Algeria,28.0339,1.6596,NaN,1/22/20,0
3,Andorra,42.5063,1.5218,NaN,1/22/20,0
4,Angola,-11.2027,17.8739,NaN,1/22/20,0


convert the date column into a `datetime` object

In [6]:
deaths['Date'] = pd.to_datetime(deaths['Date'])

sort by date

In [7]:
deaths = deaths.sort_values(by=['Date'])

In [8]:
deaths.head()

,Country/Region,Lat,Long,Province/State,Date,Deaths
0,Afghanistan,33.0000,65.0000,NaN,2020-01-22,0
167,Netherlands,12.1696,-68.9900,Curacao,2020-01-22,0
168,Netherlands,18.0425,-63.0548,Sint Maarten,2020-01-22,0
169,Netherlands,52.1326,5.2913,NaN,2020-01-22,0
170,New Zealand,-40.9006,174.8860,NaN,2020-01-22,0


In [9]:
wide = (
deaths.
groupby(['Country/Region','Date'])['Deaths'].sum().
unstack('Country/Region').
reset_index()
)

In [10]:
wide['formatted_date'] = wide['Date'].apply(lambda d: d.strftime('%Y-%m-%d'))

## Index from $N^{th}$ death

In [17]:
from bokeh.io import output_file, show
from bokeh.layouts import row
from bokeh.models import CustomJS, MultiSelect, ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import inferno

N=100 

output_file("index.html", title=f"COVID-19_analysis_from_{N}th_death")



#get list of all countries from column name, we don't want the date columns
countries = wide.columns.unique().drop(['Date', 'formatted_date'])

#store the data for each country
sources = dict.fromkeys(countries)

TOOLTIPS = [
    ("index", "$index"),
    ("(Day,Deaths)", "(@index, $y{int})"),
    ("country", "$name"),
]

#create colour palette
colours = inferno(len(countries))

p = figure(title=f"COVID-19 Deaths (from {N}th Death)",
           y_axis_type='log',
              tools="crosshair,pan,reset,save,wheel_zoom", tooltips=TOOLTIPS, 
          plot_width=1000, plot_height=600)
    
    
props = dict(line_width=4, line_alpha=0.7, visible=False)

line_glyphs = dict.fromkeys(countries)
scatter_glyphs = dict.fromkeys(countries)

for idx, country in enumerate(countries):
    df = wide[['Date', country]]
    df = df[df[country]>=N]
    df.loc[:,'index'] = np.arange(len(df))
    sources[country] = ColumnDataSource(df)
    line_glyphs[country] = p.line(x='index', y=country, source=sources[country], 
                                  name=country, color=colours[idx], **props)
    scatter_glyphs[country] = p.scatter(x='index', y=country, source=sources[country], 
                                  name=country, color=colours[idx], size=10, visible = False)


pre_selected = ['United Kingdom', 'US']
for c in pre_selected:
    line_glyphs[c].visible = True
    scatter_glyphs[c].visible = True

opts = [(str(x), str(x)) for x in line_glyphs]
multi_select = MultiSelect(title="Select multiple countries using ctrl or cmd:", value=pre_selected,
                           options=opts, height=500)




select_callback = CustomJS(args=dict(line_glyphs=line_glyphs, scatter_glyphs=scatter_glyphs,
                                     multi_select=multi_select), code="""

//console.log(multi_select.value);
//console.log(glyphs[multi_select.value]);
//glyphs[multi_select.value].visible = true

//multi_select.value.forEach(element => console.log(element));

for(var index in line_glyphs) {
  line_glyphs[index].visible = false;
  scatter_glyphs[index].visible = false;
  
}

multi_select.value.forEach(element => line_glyphs[element].visible = true);
multi_select.value.forEach(element => scatter_glyphs[element].visible = true);
""")

multi_select.js_on_change('value', select_callback)

layout = row(p, multi_select)
show(layout)